## Task manager assessment

In [ ]:
# libraries
import hashlib
from Utils import *
import pandas as pd
from pathlib import Path

#### Main Method

In [ ]:
# entry point
def main():
    currentUser = None
    allTasks = []   
    currentTask = None
    while True: 
        currentUser = OnOpen()
        # check if user exists
        userExists = CheckUserExist(currentUser)
        if userExists == True:
            taskFile=taskFileName = currentUser.Name+'_'+'TaskDB.csv'
            # get dataframe object
            df=ReadCsvTasks(taskFileName, showTasks='N')
            nOfTasks=df.shape[0]
            print(f"You logged in successfully. {currentUser.Name}  Enjoy")
            print("\nWhat would you like to do?\n1. Add a task.\n2. View Tasks.\n3. Change status.\n4. Exit.")
            consoleInput = input("Pick an option: ")
            if consoleInput == "1":
                taskId = int(nOfTasks + 1)
                taskName = input("Enter a valid name: ")
                taskDescr = input("Enter a valid description: ")
                taskStatus = "Pending"
                newTask = UserTask(taskId, taskName, taskDescr, taskStatus)
                CreateCsvTaskFile(taskFileName, newTask)
            elif consoleInput == "2":
                ReadCsvTasks(taskFileName, showTasks='Y')
            elif consoleInput == "3":
                OverrideCsv(taskFileName)
            elif consoleInput == "4":
                # dispose objects
                currentUser = None  
                currentTask = None
                print("See you next time")
                break
            else:
                print("Invalid option. Please try again.")
        else:
            print("Would you like to register?")
            consoleInput = input("Y/N:  ")
            if consoleInput == "Y":
                RegisterUser(currentUser)
                print("Thank you for registering")
            elif consoleInput == "N":
                #this.exist()
                print("See you next time")
                break
            else:
                print("Invalid option. Please try again.")
                print("See you next time")
                break
                
if __name__ == "__main__":
    main()

#### User's credentials handling

In [ ]:
## methods
# user's credentials
def HashString(inputValue):
    return hashlib.sha256(inputValue.encode()).hexdigest()
    #hashedValue = hashlib.sha256();
    #hashedValue.update(p_inputValue.encode().hexdigest())
    #return hashedValue

def RegisterUser(userArg):
    with open("registeredUsers.txt", "a") as file:
        file.write(f"{userArg.Name},{userArg.Pwd}")
    # placeholder tasks
    taskFileName = userArg.Name+'_'+'TaskDB.csv' 
    CreateCsvPlaceholder(taskFileName)
        
def Login(user):
    # Handled in Main() method  
    return
def CheckUserExist(userArg):
    with open("registeredUsers.txt", "r") as file:
        usersList = file.readlines()
        
    for user in usersList:
        # get username
        userName, userPwd = user.split(',')
        if userName == userArg.Name and userPwd == userArg.Pwd:
            return True
        return False
        
# OnOpen event
def OnOpen():
    print("Hi there. Please login into the Task Manager System\n")
    user = input("username: ")
    password = input("password: ")
    pwd = HashString(password)
    
    user = RegisteredUser(user, pwd)
    return user
    

#### Tasks handling

In [ ]:
def CreateCsvTaskFile(taskFileName, task):
    # df = pd.DataFrame(columns=['Id', 'Name', 'Description', 'Status'])
    data = {
        'Id': task.Id,
        'Name': task.Name,
        'Description': task.Description,
        'Status': task.Status
    }
    df = pd.DataFrame(data, columns=['Id', 'Name', 'Description', 'Status'], index=[0])
    my_file = Path(taskFileName)
    if my_file.is_file(): #append
        df.to_csv(taskFileName, sep='\t', mode='a', header=False)
    else: # newfile
        df.to_csv(taskFileName, sep='\t', mode='w', header=True)
        
    print(f"Tasks DB updated  {taskFileName}")

def CreateCsvPlaceholder(taskFileName):
    df = pd.DataFrame(columns=['Id', 'Name', 'Description', 'Status'], index=[0])
    my_file = Path(taskFileName)
    df.to_csv(taskFileName, sep='\t', mode='w', header=True)
    print(f"Tasks DB updated:  {taskFileName}")

def ReadCsvTasks(taskFileName, showTasks):
    my_file = Path(taskFileName)
    if my_file.is_file():
        df = pd.read_csv(taskFileName, sep='\t')
        if showTasks == 'Y':
            print("DataFrame read from CSV:")
            print(df)
        return df
    else: # newfile
        return

def EditStatusValue(dataframe, conditionColumn, conditionValue, targetColumn, newValue):
    dataframe.loc[dataframe[conditionColumn] == conditionValue, targetColumn] = newValue
    print(f"Updated {targetColumn} where {conditionColumn} is {conditionValue}")
    return dataframe

def OverrideCsv(taskFileName, conditionColumn='Id', targetColumn='Status', newValue='Completed'):
    taskId=int(input("Provide Task Id to mark as completed. n#:"))
    df = pd.read_csv(taskFileName, sep='\t')
    df = EditStatusValue(df, conditionColumn, taskId, targetColumn, newValue)
    # override csv and persist
    df.to_csv(taskFileName, sep='\t', index=False)
    print("CSV modified and persisted.")